In [ ]:

from transformers import pipeline

#set device
device = 0




# audio_file=r"C:\Users\gteja\Downloads\medical_audio_file.mp3"

pipe = pipeline(model='Bhaveen/Medical-Speech-Transcription-Whisper-Small-Fine-Tuned', device=device,return_timestamps=True)

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

c:\Users\bhave\anaconda3\envs\Medimix\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bhave\.cache\huggingface\hub\models--Bhaveen--500medimix. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
print(output['text'])

 Yes. Four is what? Costectomy, left rear assist. All right, let's do it. Thank you. What is the history? Line base negative. Levered is fine. Spleen is fine. Gall butter. No, sorry, we're going to go to FOSSA. can we call it Dectomy, pancreas is fine, thumbtack is fine, mobile loops are fine. Just add them. Add three cinema left re-assist. I'll bring it up. A very simple system. It's 15HU. Okay. Oh, before we do that. Rheumatism, erinormal cell monosidicization. IVC endomarcoval, no brainer. Opethyriacine normal. Left kidney normal, but at assist. comparison stable. Emily Sengit, chest to views. Zero, orange. Raoul, P.A.5. Left ankle. Right knee, 0-1-2. foot zero one two All fractures neck of third worth of fifth no terms of bonds Right needs a one one.


In [ ]:
from flask import Flask, render_template, request, jsonify
import os
from flask_cors import CORS
import openai
import datetime

import firebase_admin
from firebase_admin import db,storage,credentials
import datetime
import sys
from Firebase.pdf import pdf_conv
from Firebase.firebase_stuff import create_doctor_account, upload_pdf_to_bucket, download_prescription, login_firebase
#from Firebase.pharmacist import pharmacist
from Firebase.register_firebase import registration
from Firebase.text_formatting import formatting
from transformers import pipeline

device = 0
pipe = pipeline(model='Bhaveen/Medical-Speech-Transcription-Whisper-Small-Fine-Tuned', device=device,return_timestamps=True)


cred = credentials.Certificate('Firebase/credentials_new.json')
firebase_admin.initialize_app(
    cred,
    {'databaseURL' : 'https://dr-writely-default-rtdb.asia-southeast1.firebasedatabase.app/','storageBucket': 'dr-writely.appspot.com'}
)

bucket = storage.bucket()

Device set to use cuda:0


ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [43]:
# patient name -> patient phno
# date
# diagnosis
# medication
 
def get_doctor(doc_name,db):
    result = {}
    refD = db.reference(f'/User/Doctor/{doc_name}')
    refC = db.reference(f'/User/Patient')
    ch = refD.get()
    patients = [i for i in ch.keys()]
    chP = refC.get()
    for i,j in chP.items():
        if i in patients:
            for k in j:
                temp_dict = {'patient_phno': '', 'Date': '', 'Diagnosis': '', 'medication': ''}
                temp_dict['Date']=(j[k]['date'])
                temp_dict['Diagnosis']=(j[k]['diagnosis'])
                temp_dict['medication']=(j[k]['medication'])
                temp_dict['patient_phno']=(i)
                result[k] = temp_dict
    print(result)


    

In [44]:
get_doctor('Tejas',db)

{'202502012208': {'patient_phno': '9019942460', 'Date': '2025-02-01', 'Diagnosis': ' Clamydia (Chlamydia) infection  ', 'medication': [' azithromycin', ' single dose; doxycycline', ' 100 mg twice daily for 7 days  ']}}


In [87]:
def get_patient(phno,db):
    result = {}
    refP = db.reference(f'/User/Patient/{phno}')
    chP = refP.get()
    refD = db.reference(f'/User/Doctor')
    chD = refD.get()
    appts = [i for i in chP.keys()]
    print(appts)
    for i,j in chD.items():
        temp_dict = {'date':'','diagnosis':'','medication':[]}
        if phno in list(j.keys()):
            for k in appts:
                print(j[phno][k])
                if k in j[phno].keys():
                    temp_dict['date'] = j[phno][k]['date']
                    temp_dict['date'] = j[phno][k]['date']
                    temp_dict['diagnosis'] = j[phno][k]['diagnosis']
                    temp_dict['medication'] = j[phno][k]['medication']
                    result[i].append(temp_dict)
            



In [88]:
get_patient('9019942460',db)

['202502012208']
{'date': '2025-02-01', 'transcript': ' Clamydia is a bacterial infection. Clamydia trachomatis causing genital pain, discharge or no symptoms. Treatment is azithromycin, single dose or doxicycline, 100 mg twice daily for 7 days. Partial treatment is advised.'}
{}


In [103]:
#ref1 = db.reference(f'/User/Doctor/{doctor}/{patient}')
patient = '9019942460'
ref2 = db.reference(f"/User/Patient/{patient}")
refC = db.reference(f'/Creds')
ch = refC.get()
#print(ch.keys())
user = [j for i,j in ch.items() if i in [patient]]
print(user[0]['name'])
# ref3 = db.reference(f"/Pharm/{user}")
# chx = ref3.get('name')
# print(chx)

dict_keys(['9019942460', '9123456766', '9123456789', '9875684832'])
Tejas


In [104]:
def get_pharmacist(db):
    result = {}
    refPharm = db.reference('/Creds')
    ch = refPharm.get()
    for i,j in ch.items():
        result[i] = j
    return result

In [105]:
print(get_pharmacist(db))

{'9019942460': {'DOB': '30/01/2004', 'email': 'tejaslovespancakes0507@gmail.com', 'gender': 'fluid', 'name': 'Tejas', 'password': 'tejas', 'role': 'Patient'}, '9123456766': {'email': 'doctor@gmail.com', 'location': 'PES', 'name': 'reactor', 'password': 'abcd', 'role': 'doctor'}, '9123456789': {'dob': '2025-02-01', 'email': 'abc@gmail.com', 'gender': 'Fluid', 'name': 'John Doe', 'password': 'abc', 'role': 'patient'}, '9875684832': {'email': 'cock@gmail.com', 'location': 'PES', 'name': 'Jane Smith', 'password': 'abcde', 'role': 'pharmacist'}}


In [112]:
def get_doctor(doc_ph,db):
    result = {}
    refCred = db.reference(f'/Creds')
    chx = refCred.get()
    doc = [j for i,j in chx.items() if i in [doc_ph]]
    print(doc)
    doc_name = doc[0]['name']
    refD = db.reference(f'/User/Doctor/{doc_name}')
    refC = db.reference(f'/User/Patient')
    ch = refD.get()
    patients = [i for i in ch.keys()]
    chP = refC.get()
    for i,j in chP.items():
        if i in patients:
            for k in j:
                temp_dict = {'patient_phno': '', 'Date': '', 'Diagnosis': '', 'medication': ''}
                temp_dict['Date']=(j[k]['date'])
                temp_dict['Diagnosis']=(j[k]['diagnosis'])
                temp_dict['medication']=(j[k]['medication'])
                temp_dict['patient_phno']=(i)
                result[k] = temp_dict
    return result

In [113]:
print(get_doctor('9123456766',db))

[{'email': 'doctor@gmail.com', 'location': 'PES', 'name': 'Tejas', 'password': 'abcd', 'role': 'doctor'}]
{'202502012208': {'patient_phno': '9019942460', 'Date': '2025-02-01', 'Diagnosis': ' Clamydia (Chlamydia) infection  ', 'medication': [' azithromycin', ' single dose; doxycycline', ' 100 mg twice daily for 7 days  ']}, '202502012347': {'patient_phno': '9019942460', 'Date': '2025-02-01', 'Diagnosis': '\tViral fever  ', 'medication': ['\tParacetamol (500 to 650 mg)', ' Ibuprofen if needed  ']}, '202502012350': {'patient_phno': '9019942460', 'Date': '2025-02-01', 'Diagnosis': ' viral fever  ', 'medication': [' paracetamol 500-650 mg  ']}}
